In [77]:
# pip install crewai
from crewai import Agent, Task, Crew
from crewai.tools import tool
import os
from dotenv import load_dotenv

In [ ]:
# The agent asks the developer a program file. 
# {Program}{Language}
# If it's a C++ code, the agent will add the FidesZKP.h library to the user's program.cpp program and will save it as program_AddedFidesZKPLib.cpp 
# If it's a Python code, the agent will add the Fides Python package to the user's program.py program and will save it as program_AddedFidesZKPLib.py 
# 
# {Processor} 
# Then, the agent asks the developer to choose a target processor including RISC-V and ARM. 
# Then, the agent compiles the code using g++ compiler for the target processor to generate an assembly file; program_AddedFidesZKP.s
# Then, the agent executes the commitmentGenerator file 
# to create program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json, program_AddedFidesZKP_Commitment.json
# 
# The agent returns program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json files to the developer.
# The agent uploads the program_AddedFidesZKP_Commitment.json on the Fides Innova blockchain using https://rpc.fidesinnova.io API and its own wallet.
# 
# The agent return a link to the explorer to show the submitted commitment file, https://explorer.fidesinnova.io/xxxxxxxxxx
# The Fides Innova explorer shows the "CommitmentManagerAIAgent" name as the transaction submitter on the blockchain. 

In [78]:
# --- Step 2: Load environment variables from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [83]:
# Role, Goal, Backstory, LLm instance, Tool (optional)
# CompilerTool = tool()
# CompilerAgent = Agent(role="commitmentGenerator")
# CompileTask = Task()
# g++ -std=c++17 -S program.cpp -o program.s -lstdc++  

# @mydecor

# def adder():
#     print("hi")

# def mydecor(adder):
#     def inter():
#         print("how are you?")
#         adder()
#     return inter

# Role, Goal, Backstory, LLm instance, Tool (optional)
@tool("CompilerTool")
def compilerTool(program: str) -> str:
    """
    Read the user program file, and recognize the program language.
    Add new lines to the program to import the FidesZKP library or package.
    Add '_AddedFidesZKP' to the program name and save it.
    """

    filename = os.path.basename(program)
    base_name, ext = os.path.splitext(filename)
    ext = ext.lower()
    filename = base_name + ext

    if ext == ".py":
        language = "Python"
    elif ext == ".cpp":
        language = "C++"
    else:
        raise ValueError("Unsupported file extension")

    with open(program, "r") as f1:
        program_code = f1.read()

    if language == "C++":
        modified_code = '#include "lib/fidesinnova.h"\n' + program_code
        new_filename = f"{base_name}_AddedFidesZKPLib.cpp"

        with open(new_filename, "w+") as f2:
            f2.write(modified_code)

        asm_filename = f"{base_name}_AddedFidesZKPLib.s"

        command_to_execute = f"g++ -std=c++17 -S {new_filename} -o {asm_filename} -lstdc++"
        print(command_to_execute)
        
        os.system(command_to_execute)

        return f"The new assembly filename is: {asm_filename}"

    else:
        raise NotImplementedError("Only C++ is supported for now")    

CompilerAgent = Agent(role="compiler", goal = "Execute the compilerTool tool. The default {program} filename is program.cpp if it's not set by the user",
                      backstory="", tools = [compilerTool])
CompileTask = Task( agent = CompilerAgent, description = "process the user's program file {program} with compilerTool.", expected_output="Only the assembly output filename resulting from executing of the g++ compiler" )

In [85]:
# Role, Goal, Backstory, LLm instance, Tool (optional)

# deviceConfigTool = tool()
# deviceConfigAgent = Agent(role="commitmentGenerator")
# deviceConfigTask = Task()

@tool("deviceConfigTool")
def deviceConfigTool(myJson) -> str:
    """
    Save he input myJson json in the device_config.json file.
    """

    with open("device_config.json", "w+") as f:
        f.write(myJson)
        
deviceConfigAgent = Agent(role="device_config", goal = "Create a json with the following items with keyword class {class} and default value 1, with keyword deviceType {deviceType} and default value sensor, with keyword deviceIdType {deviceIdType} and default value MAC, with keyword deviceModel {deviceModel} and default value Siemens IOT2050, with keyword manufacturer {manufacturer} and default value FDS, with keyword softwareVersion {softwareVersion} and default value 1.0,  with keyword code_block {code_block} and default value [14,15]. Send the generated json to the deviceConfigTool tool.",
                      backstory="", tools = [deviceConfigTool])
deviceConfigTask = Task( agent = deviceConfigAgent, description = "Create a json with the following items with keyword class {class} and default value 1, with keyword deviceType {deviceType} and default value sensor, with keyword deviceIdType {deviceIdType} and default value MAC, with keyword deviceModel {deviceModel} and default value Siemens IOT2050, with keyword manufacturer {manufacturer} and default value FDS, with keyword softwareVersion {softwareVersion} and default value 1.0,  with keyword code_block {code_block} and default value [14,15]. Send the generated json to the deviceConfigTool tool.", expected_output="Print a message whether the device config file is updated or not." )

In [86]:
# Agent, Description, Expected Output   
agents = [CompilerAgent, deviceConfigAgent]
tasks = [CompileTask, deviceConfigTask]
crew1 = Crew( agents = agents, tasks= tasks, verbose = True)

In [87]:
def query():
    question = input("What's your program name? (default: program.cpp)")
    program_class = input("What's your program ZKP class? (default: 1)")
    deviceType = input("What's your program device type? (default: Sensor)")
    deviceIdType = input("What's your device ID type? (default: MAC)")
    deviceModel = input("What's your device model? (default: Siemens IOT2050 2.5)")
    manufacturer = input("What's your brand name? (default: Fides)")
    softwareVersion = input("What's your program/firmware version? (default: 1.0)")
    code_block = input("What's your program/firmware version? (default: [14, 15]])")

    result = crew1.kickoff({"program":question, "class":program_class, "deviceType":deviceType, "deviceIdType":deviceIdType, "deviceModel":deviceModel, "manufacturer":manufacturer, "softwareVersion":softwareVersion, "code_block":code_block })
    return result

query()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 496e0ad6-abd0-4031-b654-36d0ae8a2996                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 35ce157a-2753-4db5-ba38-81933887f7bc
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 35ce157a-2753-4db5-ba38-81933887f7bc
       Status: Executing Task...
    └── 🤖 Agent: compiler
            Status: In Progress

# Agent: compiler
## Task: process the user's program file  with compilerTool.


🤖 Agent: compiler
    Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 35ce157a-2753-4db5-ba38-81933887f7bc
│      Assigned to: compiler
│      Status: ✅ Completed
│   └── 🤖 Agent: compiler
│           Status: ✅ Completed
└── 📋 Task: b5752a48-c82e-4b27-aa52-ebdaa81cfa58
       Status: Executing Task...
    └── 🤖 Agent: device_config
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 35ce157a-2753-4db5-ba38-81933887f7bc
│      Assigned to: compiler
│      Status: ✅ Completed
│   └── 🤖 Agent: compiler
│           Status: ✅ Completed
└── 📋 Task: b5752a48-c82e-4b27-aa52-ebdaa81cfa58
       Assigned to: device_config
       Status: ✅ Completed
    └── 🤖 Agent: device_config
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b5752a48-c82e-4b27-aa52-ebdaa81cfa58                                                                     │
│  Agent: device_config                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 496e0ad6-abd0-4031-b654-36d0ae8a2996                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw='The device configuration file has been successfully updated with the JSON object as specified above.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description="process the user's program file  with compilerTool.", name=None, expected_output='Only the assembly output filename resulting from executing of the g++ compiler', summary="process the user's program file  with compilerTool....", raw='program_AddedFidesZKPLib.s', pydantic=None, json_dict=None, agent='compiler', output_format=<OutputFormat.RAW: 'raw'>), TaskOutput(description='Create a json with the following items with keyword class  and default value 1, with keyword deviceType  and default value sensor, with keyword deviceIdType  and default value MAC, with keyword deviceModel  and default value Siemens IOT2050, with keyword manufacturer  and default value FDS, with keyword softwareVersion  and default value 1.0,  with keyword code_block  and default value [14,15]. Send the generated json to the devic

In [ ]:
# Role, Goal, Backstory, LLm instance, Tool (optional)

# commitmentGeneratorTool = tool()
# commitmentGeneratorAgent = Agent(role="commitmentGenerator")
# commitmentGeneratorTask = Task()

# Ask the following parameters from developer and write in device_config.json 
#   "class": 1,
#   "deviceType": "Sensor",
#   "deviceIdType": "MAC",
#   "deviceModel": "Siemense IOT2050 2.5",
#   "manufacturer": "MHT",
#   "softwareVersion": "1.3",
# then, execute:
# commitmentGenerator-ubuntu devprogra.s
# commitmentGenerator-Mac devprogra.s


# Role, Goal, Backstory, LLm instance, Tool (optional)
@tool("commitmentGeneratorTool")

def commitmentGeneratorTool(program):
    """Read the user program__AddedFidesZKP.s file, and execute the ./commitmentGenerator file with the program__AddedFidesZKP.s file to generate three files including program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json, and program_AddedFidesZKP_Commitment.json"""
        # Outputs:
        # 1- program_AddedFidesZKP_ZKPGen.s 
        # 2- program_AddedFidesZKP_Param.json 
        # 3- program_AddedFidesZKP_Commitment.json
    if ".s" in program:
        Language = "assembly"
    else:
        Language = "Unknown"

    if Language == "assembly":
        os.system(f"./commitmentGenerator {program}")
        return "The new files are: " + program.split('.')[0]+"_ZKPGen.s," + program.split('.')[0]+"_Param.json, and "  + program.split('.')[0]+"_Commitment.json."
    else:
        raise NotImplementedError    
 
 def blockchainUploader(program):
    """Upload program_AddedFidesZKP_Commitment.json on fides Innova blockchain"""
        # program_AddedFidesZKP_Commitment.json
    if ".s" in program:
        Language = "assembly"
    else:
        Language = "Unknown"

    if Language == "assembly":
        os.system(f"./commitmentGenerator {program}")
        return "The transaction id: " + program.split('.')[0]+"_ZKPGen.s,"
    else:
        raise NotImplementedError   
    

 {
  "class": 1,
  "deviceType": "Sensor",
  "deviceIdType": "MAC",
  "deviceModel": "Siemense IOT2050 2.5",
  "manufacturer": "MHT",
  "softwareVersion": "1.3",
  "code_block": [
    14,
    15
  ]
}
    
commitmentGeneratorAgent = Agent(role="commitmentGenerator", goal="Read the assembly filename from the CompilerAgent. Then, in the device_config.json file, update class, .",
                      backstory="", tools = [commitmentGeneratorTool])
commitmentGeneratorTask = Task( agent = commitmentGeneratorAgent, description = "execute the user's program file {program} with commitmentGenerator, and upload the generated commitment file on Fides Innova blockchain", expected_output="Two files, program_AddedFidesZKP_ZKPGen.s and program_AddedFidesZKP_Param.json to be delivered to the developer. One file, program_AddedFidesZKP_Commitment.json , to be uploaded on the Fides blockchain." )



In [ ]:
# Role, Goal, Backstory, LLm instance, Tool (optional)
# commitmentSubmitterTool = tool()
# commitmentSubmitterAgent = Agent(role="commitmentSubmitter")
# commitmentSubmitterTask = Task()
@tool("commitmentSubmitterTool")

def commitmentSubmitterTool(program):
    """Read the user program__AddedFidesZKP.s file, and execute the ./commitmentGenerator file with the program__AddedFidesZKP.s file to generate three files including program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json, and program_AddedFidesZKP_Commitment.json"""
        # Outputs:
        # 1- program_AddedFidesZKP_ZKPGen.s 
        # 2- program_AddedFidesZKP_Param.json 
        # 3- program_AddedFidesZKP_Commitment.json
    if ".s" in program:
        Language = "assembly"
    else:
        Language = "Unknown"

    if Language == "assembly":
        os.system(f"./commitmentGenerator {program}")
        return "The new files are: " + program.split('.')[0]+"_ZKPGen.s," + program.split('.')[0]+"_Param.json, and "  + program.split('.')[0]+"_Commitment.json."
    else:
        raise NotImplementedError    
 
 def blockchainUploader(program):
    """Upload program_AddedFidesZKP_Commitment.json on fides Innova blockchain"""
        # program_AddedFidesZKP_Commitment.json
    if ".s" in program:
        Language = "assembly"
    else:
        Language = "Unknown"

    if Language == "assembly":
        os.system(f"./commitmentGenerator {program}")
        return "The transaction id: " + program.split('.')[0]+"_ZKPGen.s,"
    else:
        raise NotImplementedError   
    
commitmentSubmitterAgent = Agent(role="commitmentGenerator", goal="Read the user program file {program}, and execute commitmentGenerator file. It generates three output files. The firs file is program_AddedFidesZKP_ZKPGen.s that has some extra opcodes compared to the program_AddedFidesZKP.s file. These lines call the proof_generation function from the FidesZKP library or package. The second file is program_AddedFidesZKP_Param.json that contains some pre-calculated numbers to accelerate the proof generation function in production. The third one is a commitment file that shows the arithmetic version of the program, program_AddedFidesZKP_Commitment.json, and should be uploaded on the Fides blockchain. This file will be used later in verification purposes by the proof verifier crew.",
                      backstory="", tools = [commitmentGeneratorTool])
commitmentSubmitterTask = Task( agent = commitmentGeneratorAgent, description = "execute the user's program file {program} with commitmentGenerator, and upload the generated commitment file on Fides Innova blockchain", expected_output="Two files, program_AddedFidesZKP_ZKPGen.s and program_AddedFidesZKP_Param.json to be delivered to the developer. One file, program_AddedFidesZKP_Commitment.json , to be uploaded on the Fides blockchain." )



In [ ]:
# Agent, Description, Expected Output   

agents = [compilerAgent, commitmentGeneratorAgent, commitmentSubmitterAgent]
# agents = [compilerAgent]

task = [compileTask, commitmentGeneratorTask, commitmentSubmitterTask]
# tasks = [compileTask]

crew1 = Crew( agents = agents, tasks= tasks, verbose = True)
crew1.kickoff({"program":"zksensor.cpp"})